# Setting

In [1]:
# 라이브러리 불러오기
import os
import json
import yaml
import shutil
import numpy as np
import logging
from random import sample
import random
from tqdm import tqdm

In [2]:
# 시드를 설정
# 반복 수행하더라도 train, valid, test가 동일하게 분할되도록 함
random.seed(42)

In [3]:
# log 생성
logger = logging.getLogger()
logger.setLevel(logging.INFO)   
formatter = logging.Formatter('Time : %(asctime)s , Message : %(message)s')  # log 출력 형식 지정

# log를 console에 출력
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.info('preprocessing started')

Time : 2024-03-05 11:03:01,370 , Message : preprocessing started


## Function

In [4]:
# 폴더 생성 함수
def createFolder(root_dir):
    '''
    폴더 구조
    train
        images
        labels
        RS_labels
    valid
        images
        labels
        RS_labels
    test
        images
        labels
        RS_labels
    '''
    try:
        for mode in mode_list:  # Train / Valid / Test
            for IL in ['images', 'labels', 'RS_labels']:     
                    # 이미지 파일 / yolo input data형식으로 전처리한 파일 / 라벨링 파일
                    for object in obc_list:   # 식용곤충 6종
                        dir = root_dir + f'/{mode}/{IL}/{object}'
                        if not os.path.exists(dir):
                            os.makedirs(dir)
    except OSError:
        print('Error: Creating directory. ' + root_dir)

In [5]:
# TVT 분할 함수
def data_copy_save(load_path, save_path, object):
    # 파일명 목록 가져오기
    file_names = os.listdir(load_path + f'/image/{object}')

    # index를 통해 TVT 분할
    total_idx = range(len(file_names))
    train_idx = sample(total_idx, int(len(file_names) * 0.8))  # train 비율 80% 지정
    temp_idx = list(set(total_idx) - set(train_idx))    
    valid_idx = sample(temp_idx, int(len(temp_idx) * 0.5))  # train을 제외하고 50%씩 분할  
    test_idx = list(set(temp_idx) - set(valid_idx))
    tvt_list = np.asarray(file_names)

    # TVT 리스트 저장
    train_list = tvt_list[train_idx]
    valid_list = tvt_list[valid_idx]
    test_list = tvt_list[test_idx]

    for tvt_list, mode in zip([train_list, valid_list, test_list], mode_list):
        for file_name in tqdm(tvt_list, desc = f'{object} | {mode} |'):
            
            # 이미지, 라벨 복사
            shutil.copyfile(load_path + f'/image/{object}/' + file_name,
                            save_path + f'/{mode}/images/{object}/' + file_name)
            
            shutil.copyfile(load_path + f'/label/{object}/' + file_name.split('.')[0] + '_(4_1).json',
                            save_path + f'/{mode}/RS_labels/{object}/' + file_name.split('.')[0] + '_(4_1).json')  
            
            # 라벨링 파일을 yolo 모델 형식에 맞도록 변환
            # 라벨링 파일 load
            with open(load_path + f'/label/{object}/' + file_name.split('.')[0] + '_(4_1).json', encoding = 'utf-8') as f:
                jsonfile = json.load(f)
                
            image_info = jsonfile['images'][0]  # 이미지 정보는 리스트 안에 있으므로 첫 번째 항목을 선택합니다.
            w = image_info['width']    # 해상도 : 너비
            h = image_info['height']   # 해상도 : 높이
            
            box_objects = jsonfile['annotations']  # 어노테이션 부분 추출
            labels = []
            bboxs = []
            

            category_mapping = {68: 0, 67: 1, 57: 2, 33: 3}   # 일부 라벨만 학습하기 위해 변형
            for obj in box_objects:
                if obj['category_id'] in [68, 67, 57, 33] :  # apple, banana, bowl, dumbbell 일 경우에만 라벨 추가
                    labels.append(category_mapping[obj['category_id']])   # 라벨을 0,1,2,3으로 매핑해서 추가
                    bboxs.append(obj['bbox'])

            bboxs = np.asarray(bboxs, dtype=np.float64)

            try:   # BBOX의 너비와 높이를 해상도의 너비와 높이로 나누어 표준화
                bboxs[:, [0,2]] /= w   
                bboxs[:, [1,3]] /= h
                labels = np.array(labels).reshape(len(labels), 1)   # 4개 클래스가 안 들어있는 이미지를 pass하도록 try문으로 구성
                bboxs[:, :2] = bboxs[:, :2] + bboxs[:, 2:] / 2.0
                annot_out = np.concatenate((labels, bboxs), axis=1)
            except: pass

            # 변환 완료한 txt 파일 저장
            # yolo 모델은 txt 형식의 라벨링 파일이 필요함
            with open(root_dir + f'/{mode}/labels/{object}/' + file_name.split('.')[0] + '.txt', "w") as f:
                for line in annot_out:
                    line = np.asarray(line, dtype=str)
                    line[0] = line[0].split('.')[0]
                    f.write(' '.join(line) + '\n')

In [6]:
# yolo모델에 필요한 yaml 파일 생성 함수
def Yolov8_yaml(classes, root_dir):
    info = {
        'train' : './train/images',
        'val': './valid/images',
        'test': './test/images',
        'nc' : len(classes),
        'names' : classes
        }
    
    # 파일 저장
    with open(f'{root_dir}/Yolov8_data.yaml', 'w') as f:
        yaml.dump(info, f)

## Run

In [7]:
# 원천 데이터 경로
original_data_path = './subset_data'      

# 저장할 TVT 데이터 경로
root_dir = './subset_data_TVT'                  

# Train, Valid, Test
mode_list = ['train', 'valid', 'test']

# 학습할 객체 폴더명
obc_list = ['apple', 'banana','bowl','dumbbell']

In [8]:
classes = [
    'apple',
    'banana',
    'bowl',
    'dumbbell'
]

In [9]:
# directory 생성
createFolder(root_dir = root_dir)

# yolov8 data yaml file 생성
Yolov8_yaml(classes=classes, root_dir = root_dir) 

logger.info('folder and yaml file created')

Time : 2024-03-05 11:03:01,469 , Message : folder and yaml file created


In [10]:
# TVT 분할
for object in obc_list:          
        data_copy_save(load_path = original_data_path, save_path = root_dir, 
                       object = object)
        
        logger.info(f'{object} TVT split completed')

logger.info('preprocessing ended')

apple | train |: 100%|██████████| 3873/3873 [00:21<00:00, 182.74it/s]
apple | valid |: 100%|██████████| 484/484 [00:02<00:00, 180.92it/s]
apple | test |: 100%|██████████| 485/485 [00:02<00:00, 188.42it/s]
Time : 2024-03-05 11:03:27,936 , Message : apple TVT split completed
banana | train |: 100%|██████████| 13526/13526 [01:16<00:00, 175.73it/s]
banana | valid |: 100%|██████████| 1691/1691 [00:10<00:00, 166.88it/s]
banana | test |: 100%|██████████| 1691/1691 [00:10<00:00, 166.48it/s]
Time : 2024-03-05 11:05:05,236 , Message : banana TVT split completed
bowl | train |: 100%|██████████| 3952/3952 [00:24<00:00, 160.93it/s]
bowl | valid |: 100%|██████████| 494/494 [00:02<00:00, 171.64it/s]
bowl | test |: 100%|██████████| 495/495 [00:03<00:00, 144.09it/s]
Time : 2024-03-05 11:05:36,206 , Message : bowl TVT split completed
dumbbell | train |: 100%|██████████| 13936/13936 [01:23<00:00, 166.32it/s]
dumbbell | valid |: 100%|██████████| 1742/1742 [00:10<00:00, 171.72it/s]
dumbbell | test |: 100%|